Converting raw `html` to `vdom`.

In [2]:
import html, vdom.svg, functools, inspect

`dom` are a collection or existing `vdom` tags; if we can't find a tag then we make our won `vdom.createComponent`.

In [3]:
dom = {**vars(vdom.svg), **vars(vdom)}

`get_tag` will return the `'tag_name'` inside of a `functools.partial`.

In [4]:
def get_tag(partial):
    if not callable(partial): return None
    def get_func(f): return get_func(f.func) if hasattr(f, 'func') else f
    return inspect.getclosurevars(get_func(partial)).nonlocals['tag_name']

In [11]:
from html.parser import HTMLParser

class vdomParser(HTMLParser):
    def feed(self, str, tag=None):
        self.buf, self.final, leftover = [], [], []
        super().feed(str)
        while self.buf: leftover=[self.buf.pop()(*leftover)]
        leftover and self.final.append(*leftover)    
        if len(self.final) == 1:
            return self.final[0]
        return (tag or vdom.div)(*self.final)
    
    def handle_starttag(self, tag, attrs):
        self.buf.append(functools.partial(dom.get(tag, vdom.create_component(tag)), **dict(attrs)))

    def handle_endtag(self, tag):
        if self.buf:
            object = self.buf.pop()
            if tag == get_tag(object):
                if self.buf: 
                    self.buf[-1] = functools.partial(self.buf[-1], object())
                else: 
                    self.final.append(object())
            else:
                self.buf.append(functools.partial(object, dom.get(tag, vdom.create_component(tag))()))
        else: self.final.append(F"</{tag}>")
        

    def handle_data(self, data):
        if self.buf: 
            if callable(self.buf[-1]): self.buf[-1] = functools.partial(self.buf[-1], data)
            else: self.buf.append(data)
        else: self.final.append(data)

parser = vdomParser()

In [6]:
    sample = vdom.div(
        vdom.h1(vdom.code('vdom'), ' is sweet.'),
        vdom.p('I don''t think there is a raw ', vdom.code('html'), ' object to ', vdom.code('vdom'), ' yet.'),
        vdom.p('This post attempts to think about how that is possible')
    )

In [7]:
    def test_transform():
        assert parser.feed(
            sample._repr_html_()
        )._repr_html_() == sample._repr_html_()

In [8]:
    def test_others():
        parser.feed("<h1>testing</p></h1>")

In [10]:
    if __name__ == '__main__':
        !ipython -m pytest -- 2018-11-01-vdom-from-raw-html.ipynb

============================= test session starts =============================
platform win32 -- Python 3.6.6, pytest-3.5.1, py-1.5.3, pluggy-0.6.0
Matplotlib: 2.2.2
Freetype: 2.8.1
rootdir: C:\Users\deathbeds\deathbeds.github.io\deathbeds, inifile: tox.ini
plugins: xonsh-0.8.1, xdist-1.22.5, testmon-0.9.12, remotedata-0.2.1, parallel-0.0.2, openfiles-0.3.0, mpl-0.9, localserver-0.4.1, forked-0.2, doctestplus-0.1.3, arraydiff-0.2, hypothesis-3.66.16, importnb-0.5.0
collected 2 items

2018-11-01-vdom-from-raw-html.ipynb F.                                   [100%]

================================== FAILURES ===================================
_______________________________ test_transform ________________________________

    def test_transform():
        assert parser.feed(
            sample._repr_html_()
>       )._repr_html_() == sample._repr_html_()
E       AssertionError

2018-11-01-vdom-from-raw-html.ipynb:125: AssertionError
===================== 1 failed, 1 passed in 0.60 seco

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2788: UserWarning: Unknown failure executing module: <pytest>
  warn('Unknown failure executing module: <%s>' % mod_name)
